In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Title

title = widgets.HTML('<h1 style="color:Green"><center> Welcome to Mask Creator </center></h1>')
title

HTML(value='<h1 style="color:Green"><center> Welcome to Mask Creator </center></h1>')

In [3]:
# Upload
selectMsg = widgets.HTML('<strong> Please select the file to upload: </strong>', layout={'padding': '0 600px 0 0'})

upload = widgets.FileUpload(accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
                            multiple=True  # True to accept multiple files upload else False
                          )
uploadPane = widgets.HBox([selectMsg, upload])
uploadPane

In [4]:
# upload.metadata[0]['name']
# upload.multiple

In [5]:
# Create slider
sourceFolder = widgets.Text(value=os.getcwd(), placeholder='Path', description='Path', disabled=False)
sourceFolder.layout = {'width':'65%'}
textBox = widgets.Text(value='test.jpg', placeholder='Filename', description='Filename:',disabled=False)
entirePath = widgets.HBox([sourceFolder, textBox])
# widgets.jslink((textBox,'value'), (upload, 'data'))

low_hue = widgets.IntSlider(min=0, max=179, step=1, description='Low_Hue', layout={'width':'40%'})
low_sat = widgets.IntSlider(min=0, max=255, step=1, description='Low_Sat', layout={'width':'40%'})
low_val = widgets.IntSlider(min=0, max=255, step=1, description='Low_Val', layout={'width':'40%'})
lowerScale = widgets.HBox([low_hue, low_sat, low_val])

upp_hue = widgets.IntSlider(value=179, min=0, max=179, step=1, description='Upp_Hue', layout={'width':'40%'})
upp_sat = widgets.IntSlider(value=255, min=0, max=255, step=1, description='Upp_Sat', layout={'width':'40%'})
upp_val = widgets.IntSlider(value=255, min=0, max=255, step=1, description='Upp_Val', layout={'width':'40%'})
upperScale = widgets.HBox([upp_hue, upp_sat, upp_val])

meter = widgets.VBox([entirePath, lowerScale, upperScale])

meter

In [6]:
def on_upload(change):
    filename = upload.metadata[0]['name']
    display(widgets.HTML(f'<h5><center> Successfully Uploaded: {filename} </center></h5>'))
    textBox.value = filename  # upload.metadata[0]['name']
#     upload.disabled = True
    
upload.observe(on_upload, 'data')

In [7]:
def show(imgRGB, mask, result):
    plt.figure(figsize=(20, 20))
    plt.subplot(131), plt.imshow(imgRGB), plt.title('Original', fontsize = 30)
    plt.subplot(132), plt.imshow(mask, cmap='gray'), plt.title('Mask', fontsize = 30)
    plt.subplot(133), plt.imshow(result), plt.title('Mask Applied', fontsize = 30)
    plt.tight_layout()
    plt.show

In [8]:
def smooth(photo):
    # kernel for erosion and dilation
    kernel = np.ones((3,3),np.uint8)
    
    erode = cv2.erode(photo,kernel,iterations = 1)
    dilate = cv2.dilate(erode,kernel,iterations = 1)
    return dilate    

In [9]:
def createMask(filename, 
               low_hue, low_sat, low_val, 
               upp_hue, upp_sat, upp_val, plot=True):
    
#     filename = textBox
    # reading the file as image
    image = cv2.imread(filename)
    # converting into RGB format for visualization
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # converting into HSV format
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # creating the bound for HSV value
    lower = np.array([low_hue, low_sat, low_sat])
    upper = np.array([upp_hue, upp_sat, upp_val])
    
    # creating mask 
    mask = cv2.inRange(hsv, lower, upper)
    mask = smooth(mask)
    
    # mask when applied to image
    result = cv2.bitwise_and(imgRGB, imgRGB, mask=mask)
    result = smooth(result)
    
    # for visualization
    if plot == True:
        show(imgRGB, mask, result)
    
    return mask, result
    
masking = widgets.interactive_output(createMask,
                              {'filename':textBox,
                               'low_hue':low_hue, 'low_sat':low_sat, 'low_val':low_val,
                               'upp_hue':upp_hue, 'upp_sat':upp_sat, 'upp_val':upp_val})

# display(meter, masking)

In [10]:
nextFile = widgets.Button(description='Next File',
                          disabled=False,
                          button_style='', # 'success', 'info', 'warning', 'danger' or ''
                          tooltip='Next File',
                          icon='arrow-circle-right' # (FontAwesome names without the `fa-` prefix)
                         )

saveMask = widgets.Button(description='Save Mask',
                          disabled=False,
                          button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                          tooltip='Save Mask',
                          icon='floppy-o' # (FontAwesome names without the `fa-` prefix)
                         )

saveROI = widgets.Button(description='Save ROI',
                          disabled=False,
                          button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
                          tooltip='Background Removed',
                          icon='floppy-o' # (FontAwesome names without the `fa-` prefix)
                         )

clear = widgets.Button(description='Clear',
                          disabled=False,
                          button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                          tooltip='Clear',
                          icon='eraser' # (FontAwesome names without the `fa-` prefix)
                         )

saveBoxes = widgets.HBox([nextFile, saveMask, saveROI, clear], layout={'width':'100%', 
                                                                'padding':'0px 0px 0px 600px', # top right bottom left 
                                                                'align':'right'}) 
saveBoxes

In [11]:
clickCount = 0

def clickNextFile(b):
    try:
        global clickCount
        textBox.value = upload.metadata[clickCount]['name']
        print(f'On Canvas {textBox.value}.', end='\r')
        clickCount += 1
    except:
        clickCount = 0
        print('Swiped all files thoroughly. Stopping the execution. Use "Clear" button to reuse!')
        nextFile.disabled = True

    
def clickSaveMask(b):
    mask, _ = createMask(textBox.value,
                         low_hue.value, low_sat.value, low_val.value,
                         upp_hue.value, upp_sat.value, upp_val.value, plot=False)
    
    # converting to one channel and saving the file
    height, width = mask.shape
    mask.resize(height, width, 1)
    thresholding = np.where(mask < 100, 0, 255) # changing the array into two categories 0: background 1: object
    name, ext = os.path.splitext(textBox.value)
    if os.path.isdir("../Masks/") != True:
        os.makedirs('../Masks/')
    cv2.imwrite('../Masks/' + name +'.png', thresholding)
    print(f'Successfully saved {textBox.value} in folder "Masks".', end="\r")

def clickSaveROI(b):
    _, result = createMask(textBox.value,
                         low_hue.value, low_sat.value, low_val.value,
                         upp_hue.value, upp_sat.value, upp_val.value, plot=False)
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    if os.path.isdir("../ROI/") != True:
        os.makedirs('../ROI/')
    cv2.imwrite('../ROI/' + textBox.value, result)
    print(f'Successfully saved {textBox.value} in folder "ROI".', end="\r")

def clickClear(b):
#     upload._counter = 0
    clear_output()
    %run maskCreator_2.ipynb

nextFile.on_click(clickNextFile)
saveMask.on_click(clickSaveMask)
saveROI.on_click(clickSaveROI)
clear.on_click(clickClear)

In [12]:
footer = widgets.HTML('<h5 style="color:Grey"><left> --- Temporary Logs --- </center></h1>')
footer

HTML(value='<h5 style="color:Grey"><left> --- Temporary Logs --- </center></h1>')

# Final and Display

In [13]:
final = widgets.VBox([title,
                      uploadPane,
                      meter,
                      masking,
                      saveBoxes,
                      footer,
                     ])

In [14]:
display(final)

# Yayyyyyy!

In [15]:
# for requirements.txt
# import watermark

In [16]:
# %load_ext watermark
# %watermark -v -m -p ipywidgets,IPython,cv2,numpy,matplotlib

In [17]:
# os.getcwd()

In [18]:
# files = next(os.walk(os.getcwd()))[2]
# files

In [19]:
# dirs = os.listdir()

In [20]:
# type(dirs)

In [21]:
# for file in files:
#     print(len(files))